In [1]:
%load_ext autoreload
%autoreload 2

# (a)

In [2]:
from scripts.bpe_tokenizer.owt.train_bpe_expts_owt import load_openwebtext_tokenizer
from scripts.bpe_tokenizer.tinystories.train_bpe_tinystories import load_tinystories_tokenizer

ts_tokenizer = load_tinystories_tokenizer()
owt_tokenizer = load_openwebtext_tokenizer()

In [3]:
from scripts.data.paths import OWT_VALID_FILE, TS_VALID_FILE


def load_document_texts(input_file: str) -> list[str]:
    with open(input_file) as f:
        text = f.read()

    document_texts = text.split("<|endoftext|>")
    return document_texts


ts_document_texts = load_document_texts(TS_VALID_FILE)
owt_document_texts = load_document_texts(OWT_VALID_FILE)

In [4]:
import random
import time
from collections.abc import Callable, Sequence


def calc_compression_ratio(encode_fn: Callable[[str], Sequence[int]], texts: list[str]) -> float:
    total_num_bytes = sum(len(text.encode("utf-8")) for text in texts)
    total_num_tokens = sum(len(encode_fn(text)) for text in texts)
    return total_num_bytes / total_num_tokens


ts_document_texts_sample = random.sample(ts_document_texts, k=100)
owt_document_texts_sample = random.sample(owt_document_texts, k=100)

print(calc_compression_ratio(ts_tokenizer.encode, ts_document_texts_sample))
print(calc_compression_ratio(owt_tokenizer.encode, owt_document_texts_sample))
print(calc_compression_ratio(ts_tokenizer.encode, owt_document_texts_sample))

4.081186193485658
4.364550111963086
3.2409742517383857


# (c)

In [5]:
from collections.abc import Callable, Sequence
from functools import partial


def estimate_tokenizer_throughput(encode_fn: Callable[[str], Sequence[int]], text: str) -> float:
    """Estimates bytes/second throughput of the tokenizer"""
    start_time = time.time()
    encode_fn(text)
    end_time = time.time()

    time_elapsed = end_time - start_time

    num_bytes = len(text.encode("utf-8"))

    return num_bytes / time_elapsed


# tiktoken_tokenizer.encode(allowed_special=)

print(bytes_per_second := estimate_tokenizer_throughput(tokenizer.encode, text))
print(estimate_tokenizer_throughput(partial(tiktoken_tokenizer.encode, allowed_special={"<|endoftext|>"}), text))

NameError: name 'tokenizer' is not defined

In [18]:
pile_num_bytes = 825 * (2**30)

pile_seconds = pile_num_bytes / bytes_per_second

SECONDS_PER_HOUR = 60 * 60

pile_hours = pile_seconds / SECONDS_PER_HOUR

pile_hours

49.40245097296382